In [ ]:
# !pip install biobss --upgrade

In [2]:
from scipy.signal import butter, iirnotch, lfilter
from os import listdir
from os.path import isfile, join, isdir
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
import scipy.ndimage
import scipy.stats as stats

import csv
import pickle
import pandas as pd

import biobss

from tqdm import tqdm



c:\Users\riche\AppData\Local\Programs\Python\Python39\lib\site-packages\antropy\fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit('float64(float64[:], int32)')


In [ ]:
#Filtered signal
def filter_signal(eda, sampling_rate):
    eda_f = biobss.edatools.filter_eda(eda, sampling_rate)

    plt.figure(figsize=(12,4))
    plt.plot(t,eda_f)
    plt.xlabel('$Time (s)$') 
    plt.ylabel('$EDA$')
    return eda_f

In [ ]:
eda = biobss.edatools.eda_decompose(eda_f, sampling_rate=sampling_rate)
tonic = eda.iloc[:,0]
phasic = eda.iloc[:,1]
print(eda)

In [ ]:
zero_pad = 55*sampling_rate-1
phasic = np.pad(phasic, (zero_pad, 0))
phasic = np.insert(phasic,0,1)
phasic = np.pad(phasic, (5, 0))
t2 = np.arange(0,phasic.size*(1/sampling_rate),(1/sampling_rate))
print(phasic)
print(phasic.size)
print(t2.size)

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(t2,phasic,label='phasic')
#plt.plot(t2,tonic,label='tonic')
plt.xlabel('$Time (s)$') 
plt.ylabel('$EDA$')

In [ ]:
peaks, properties = scipy.signal.find_peaks(phasic)
heights, left_bases, right_bases = scipy.signal.peak_prominences(phasic, peaks)
widths, width_heights, left_ips, __ = scipy.signal.peak_widths(phasic, peaks, rel_height=0.9)
__, __, __, right_ips = scipy.signal.peak_widths(phasic, peaks, rel_height=0.5)

heights1 = np.delete(heights, np.where(heights == np.max(heights)))
heights2 = np.delete(heights, np.where(heights1 == np.max(heights1)))

# find the indices with an amplitude larger that 0.1
keep = np.full(len(peaks), True)
amplitude_min=0.25*np.max(heights2)
time_min=50
keep[heights<amplitude_min] = False

# only keep those 
peaks=peaks[keep]
heights=heights[keep]
widths=widths[keep]
left_bases = left_bases[keep]
right_bases = right_bases[keep]
width_heights = width_heights[keep]
left_ips = left_ips[keep]
right_ips = right_ips[keep]

#keep peaks inside a certain time interval
keep1 = np.full(len(peaks), True)
peak_diff = [0]
for i in range(peaks.size-1):
  peak_diff_enum = t[peaks[i+1]]-t[peaks[i]]
  peak_diff.append(peak_diff_enum)

keep1[np.array(peak_diff)>time_min] = False

peaks=peaks[keep1]
heights=heights[keep1]
widths=widths[keep1]
left_bases = left_bases[keep1]
right_bases = right_bases[keep1]
width_heights = width_heights[keep1]
left_ips = left_ips[keep1]
right_ips = right_ips[keep1]

# plot
plt.figure(figsize=(12,4))
plt.plot(t2,phasic,label='phasic')
plt.plot(t2[peaks],phasic[peaks],'o',label='peaks')
# labels and titles
plt.xlabel('$Time (s)$') 
plt.ylabel('$EDA$')
plt.legend()

In [ ]:
#Taking the time interval between of the cluster of peaks
t2 = t[peaks[1]:peaks[-1]]
phasic = phasic[peaks[1]:peaks[-1]]
print(t2.size)
print(phasic.size)
# plot
plt.figure(figsize=(12,4))
plt.plot(t2,phasic,label='phasic')
# labels and titles
plt.xlabel('$Time (s)$') 
plt.ylabel('$EDA$')
plt.legend()

In [ ]:
#Features from EDA signal
tonic_phase_feat = biobss.edatools.from_decomposed(phasic, tonic, sampling_rate)
eda_feat = biobss.edatools.from_signal(eda_f, sampling_rate)
eda_freq_feat = biobss.edatools.eda_freq_features(eda)
tonic_feat = biobss.edatools.from_scl(tonic)

#Features from phasic
phasic_feat = biobss.edatools.from_scr(phasic)

In [ ]:
print(tonic_phase_feat)

In [ ]:
#field_names= ['scr_mean', 'scr_std', 'scr_max', 'scr_min', 'scr_range', 'scr_kurtosis', 'scr_skew', 'scr_momentum', 'scr_activity', 'scr_complexity', 'scr_mobility', 'scr_rms', 'scr_acr_length', 'scr_integral', 'scr_average_power',
 #             'scr_f1sc', 'scr_f2sc', 'scr_f3sc', 'scr_Energy', 'scr_Entropy', 'scr_max_freq', 'scl_mean', 'scl_std', 'scl_max', 'scl_min', 'scl_range', 'scl_kurtosis', 'scl_skew', 'scl_momentum']

In [ ]:
############ lowpass filtering
# Parameters
order = 4
frequency = 5
sampling_rate = 100
frequency = frequency / (sampling_rate / 2)  # Normalize frequency to Nyquist Frequency (Fs/2).

In [ ]:
#eda_data = np.empty((0,196639)) 
#eda_time_data = np.empty((0,196639)) 
#
#for i in range(56):
#  sub_ind = i
#  # read eda file
#  csvpath = dataset_path+'/'+onlydirs[sub_ind]+'/BitalinoGSR.txt'
#  # load the txt file
#  data = np.loadtxt(csvpath,dtype='str')
#  # eda recordings
#  eda = data[:,0].astype(float)[:196639][np.newaxis] # slice all the data to the length of the shortest one
#  eda_data = np.append(eda_data,eda, axis = 0)
#  # timestamps
#  eda_time = data[:,1].astype(float)[:196639][np.newaxis] # slice all the data to the length of the shortest one
#  eda_time_data = np.append(eda_time, eda_time_data, axis = 0)



In [ ]:
# Filtering
b, a = scipy.signal.butter(order, frequency)
eda_lp = scipy.signal.filtfilt(b, a, eda)

################################### Smoothing
# hybrid method
# step 1
size= int(0.75 * sampling_rate)
eda_sm0 = scipy.ndimage.uniform_filter1d(eda_lp, size, mode='nearest')

# step 2
# window
kernel="parzen"
window = scipy.signal.get_window(kernel, size)
w = window / window.sum()

eda_sm_l= np.empty((0,196639)) 

# Extend signal edges to avoid boundary effects.
signal_edge_l = eda_sm0[:0]
signal_edge_r = eda_sm0[:-1]

for i in range(15):
  pad_l = np.full(shape=38, fill_value= signal_edge_l[i], dtype= float) # pad with 38 times the first signal value
  pad_r = np.full(shape=38, fill_value= signal_edge_r[i], dtype= float) # pad with 38 times the last signal value

  eda_sm0_pad = np.concatenate((pad_l, eda_sm0[i], pad_r)) # pad the signal at begin and end
 
  # Compute moving average.
  eda_sm = np.convolve(w, eda_sm0_pad)
  eda_sm = eda_sm[size:-size]
  eda_sm_l = np.append(eda_sm_l, eda_sm[np.newaxis], axis = 0)


In [ ]:
################## decompose 
# Electrodermal Activity (EDA) into Phasic and Tonic components.
# Phasic
order=5
freqs=[0.05 / (sampling_rate)]
sos = scipy.signal.butter(order, freqs, btype='highpass', output="sos")
phasic= scipy.signal.sosfiltfilt(sos, eda_sm_l)

#Tonic
sos = scipy.signal.butter(order, freqs, btype='lowpass', output="sos")
tonic= scipy.signal.sosfiltfilt(sos, eda_sm_l)

In [ ]:
#fig, ax = plt.subplots(56, 1, figsize=(20, 100))
#
#for i in range(56):
#  eda_data_plot = eda_data[i]
#  eda_sm_plot = eda_sm_l[i]
#  phasic_plot = phasic[i]
#  tonic_plot = tonic[i]
#  t=np.arange(0,eda_time_data[i].size*0.01,0.01)
#  t=t[:eda.size]
#
#  ax[i].plot(t,eda_sm_plot)
#  ax[i].plot(t,phasic_plot)
#  ax[i].plot(t,tonic_plot)
#  # labels and titles
#  ax[i].set_xlabel('$Time (s)$') 
#  ax[i].set_ylabel('$EDA$') 
#
#  # saving the figure as png
#  #plt.savefig('eda.png', dpi=300)

In [ ]:
onset_l = np.empty((0,56)) 
hrt_time_l = np.empty((0,56)) 

for i in range(56): 
  peaks, properties = scipy.signal.find_peaks(phasic[i])
  heights, left_bases, right_bases = scipy.signal.peak_prominences(phasic[i], peaks)
  widths, width_heights, left_ips, __ = scipy.signal.peak_widths(phasic[i], peaks, rel_height=0.9)
  __, __, __, right_ips = scipy.signal.peak_widths(phasic[i], peaks, rel_height=0.5)

  # find the indices with an amplitude larger that 0.1
  keep = np.full(len(peaks), True)
  amplitude_min=0.1
  keep[heights<amplitude_min] = False

  # only keep those 
  peaks=peaks[keep]
  heights=heights[keep]
  widths=widths[keep]
  left_bases = left_bases[keep]
  right_bases = right_bases[keep]
  width_heights = width_heights[keep]
  left_ips = left_ips[keep]
  right_ips = right_ips[keep]

  onset = left_ips.astype(int) / sampling_rate
  onset_index = left_ips.astype(int)
  onset_l = np.append(onset_l,onset[:56][np.newaxis], axis = 0)
  #print("Onset time is\n",onset)

  hrt = right_ips.astype(int) / sampling_rate
  hrt_index = right_ips.astype(int)
  hrt_time = (hrt_index - peaks) / sampling_rate
  hrt_time_l = np.append(hrt_time_l,hrt_time[:56][np.newaxis], axis = 0)
  #print("Half recovery time is\n",hrt_time)

  ### plot
  #t=np.arange(0,eda_time_data[i].size*0.01,0.01)
  #t=t[:eda.size]
#
  #plt.figure(figsize=(25,2))
  #plt.plot(t,phasic[i],label='phasic')
  #plt.plot(t[peaks],phasic[i,peaks],'o',label='peaks')
  #plt.plot(onset, phasic[i,onset_index],'x',label='onset')
  #plt.plot(hrt, phasic[i,hrt_index],'x',label='half recovery')
  ## labels and titles
  #plt.xlabel('$Time (s)$') 
  #plt.ylabel('$EDA$') 
  #plt.legend()

In [ ]:
mean = np.mean(hrt_time_l, axis= 1)
min = np.amin(hrt_time_l, axis= 1)
max = np.amax(hrt_time_l, axis= 1)
print(mean)
print(min)
print(max)